In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

In [20]:
# Import the dataset
white_wine_data=pd.read_csv('wine+quality/winequality-white.csv',sep=';')
red_wine_data=pd.read_csv('wine+quality/winequality-red.csv',sep=';')

print(white_wine_data.shape)
print(red_wine_data.shape)

(4898, 12)
(1599, 12)


## Question 4
Let’s do a linear regression.  
• Do the variable transformations that are needed (on hot encoding, log etc...)  
• Fit the linear regression  
• Interpret the coefficients of your regression  
• Does any coefficient contradict your intuition?  


### Linear Regression model

In [21]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


#We started by creating our model with learning Regressio,
model = LinearRegression()

#We then split the data into the input and output
X = white_wine_data.drop('quality', axis=1)
y = white_wine_data['quality']
model.fit(X, y)

#Print the coefficients
print("coef : ", model.coef_)
print("intercept_ : ",model.intercept_)

#Then we make the prediction
y_pred = model.predict(X)

#Calculate the mean squared error
mse = mean_squared_error(y, y_pred)
print("Mean scare error :", mse)

print(white_wine_data.head(5))

coef :  [ 6.55199614e-02 -1.86317709e+00  2.20902007e-02  8.14828026e-02
 -2.47276537e-01  3.73276519e-03 -2.85747419e-04 -1.50284181e+02
  6.86343742e-01  6.31476473e-01  1.93475697e-01]
intercept_ :  150.19284248121488
Mean scare error : 0.563154062988657
   fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0            7.0              0.27         0.36            20.7      0.045   
1            6.3              0.30         0.34             1.6      0.049   
2            8.1              0.28         0.40             6.9      0.050   
3            7.2              0.23         0.32             8.5      0.058   
4            7.2              0.23         0.32             8.5      0.058   

   free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
0                 45.0                 170.0   1.0010  3.00       0.45   
1                 14.0                 132.0   0.9940  3.30       0.49   
2                 30.0                  97.0   0.99

### Interpretation



L'intercept est la valeur prédite de la variable de sortie lorsque toutes les variables d'entrée sont nulles. Il représente le point de départ de la ligne de régression.
Ici sa valeur est de 150? ceci n'a pas de sens 

Pour les coefficients, ceci indique comment les variables affectent la variable de sortie. Un coefficient indique une relation positive tandis qu'un coefficient négatif indique une relation négative.